In [1]:
# tensorboard --logdir=logs
# http://desktop-bu5qmic:6006/

In [2]:
from __future__ import print_function

import argparse
import os
import random
import sys

import numpy as np

In [3]:
import shutil

# make or reset directory
def mk_reset_dir(directory):
    if os.path.exists(directory):
        try:
            shutil.rmtree(directory)
            os.mkdir(directory)
        except:
            print("error:", directory)
    else:
        try:
            os.mkdir(directory)
        except:
            print("error create:", directory)

In [4]:
# getting parent directory
project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [6]:
# # setting data path data seperated by class name
# training_dir = "data\\input\\train"
# testing_dir = "data\\input\\test"
# validation_dir = "data\\input\\validation"

# # joining path with parent path 
# training_dir = os.path.join(project_dir, training_dir)
# testing_dir = os.path.join(project_dir, testing_dir)
# validation_dir = os.path.join(project_dir, validation_dir)


# # setting output directory
# model_dir =  "data\\output\\models"
# log_dir = "data\\output\\logs"

# # joining path with parent path 
# model_dir = os.path.join(project_dir, model_dir)
# log_dir = os.path.join(project_dir, log_dir)


In [36]:
training_dir = r'data/training'
validation_dir = r'data/validation'
testing_dir = r'data/test'
model_dir = 'output/models/'
log_dir = "output/logs"

model_file = model_dir+"weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

In [37]:
from keras import metrics

# setting training parameters
norm = 255.0
rescale=1./norm
shear_range=0.2
zoom_range=0.2
horizontal_flip=True

# setting train, test, validation parameters
target_size=(224, 224)
batch_size=32
class_mode='categorical'

loss='categorical_crossentropy'
metrics=['accuracy']

epochs = 50
verbose = 1

In [38]:
mk_reset_dir(model_dir)
mk_reset_dir(log_dir)

error: output/logs


In [8]:
# model_file = "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
# model_file = os.path.join(model_dir, model_file)

In [9]:
model_file

'output/models/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'

In [39]:
from keras.preprocessing.image import ImageDataGenerator

# data generator for training
train_datagen = ImageDataGenerator(
        rescale=rescale,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip)

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

# data generator for validation
validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

# # data generator for testing
# test_datagen = ImageDataGenerator(rescale=rescale)
# test_generator = test_datagen.flow_from_directory(
#         testing_dir,
#         target_size=target_size,
#         batch_size=batch_size,
#         class_mode=class_mode)

Found 1097 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [40]:
from keras.applications.inception_v3 import InceptionV3

# importing inception model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [41]:
from keras.layers import Dense, GlobalAveragePooling2D

# setting model layers specially output layer with class number
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)


In [42]:
from keras.models import Model

#loading model
model = Model(inputs=base_model.input, outputs=predictions)

In [43]:
# set all laeyrs as untrainable
for layer in base_model.layers:
    layer.trainable = False

In [44]:
from keras import optimizers

# chose and configure optimizer 
sgd = optimizers.Adam()
# sgd = optimizers.SGD()
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = optimizer=SGD(lr=0.0001, momentum=0.9)

In [45]:
# compile model with optimizer and loss
model.compile(sgd, loss=loss, metrics=metrics)

In [46]:
#print layers of inception model
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_2
1 conv2d_95
2 batch_normalization_95
3 activation_95
4 conv2d_96
5 batch_normalization_96
6 activation_96
7 conv2d_97
8 batch_normalization_97
9 activation_97
10 max_pooling2d_5
11 conv2d_98
12 batch_normalization_98
13 activation_98
14 conv2d_99
15 batch_normalization_99
16 activation_99
17 max_pooling2d_6
18 conv2d_103
19 batch_normalization_103
20 activation_103
21 conv2d_101
22 conv2d_104
23 batch_normalization_101
24 batch_normalization_104
25 activation_101
26 activation_104
27 average_pooling2d_10
28 conv2d_100
29 conv2d_102
30 conv2d_105
31 conv2d_106
32 batch_normalization_100
33 batch_normalization_102
34 batch_normalization_105
35 batch_normalization_106
36 activation_100
37 activation_102
38 activation_105
39 activation_106
40 mixed0
41 conv2d_110
42 batch_normalization_110
43 activation_110
44 conv2d_108
45 conv2d_111
46 batch_normalization_108
47 batch_normalization_111
48 activation_108
49 activation_111
50 average_pooling2d_11
51 conv2d_107
52 conv2d_109
53 co

In [47]:
# train the top 2 inception blocks 
# freeze first 249 layers
# unfreeze the rest

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [48]:
import keras

checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
# checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc')

In [49]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)


# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None)
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

In [50]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
# tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=32, write_graph=True, write_grads=False, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

tensorboard.set_model(model) 

In [51]:
# callbacks_list = [checkpoint, tensorboard, early_stopping]
callbacks_list = [checkpoint, tensorboard]

In [52]:
# train inception model
# fine-tuning the top 2 inception blocks and top Dense layers
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks_list)

C:\Users\Andromeda\Anaconda3\lib\site-packages\keras\engine\training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/50
35/35 [==============================] - 452s 13s/step - loss: 1.3737 - acc: 0.5936 - val_loss: 0.5100 - val_acc: 0.8419
Epoch 2/50
35/35 [==============================] - 457s 13s/step - loss: 0.5434 - acc: 0.8254 - val_loss: 0.4045 - val_acc: 0.8750
Epoch 3/50
35/35 [==============================] - 422s 12s/step - loss: 0.3881 - acc: 0.8754 - val_loss: 0.1974 - val_acc: 0.9338
Epoch 4/50
35/35 [==============================] - 476s 14s/step - loss: 0.3545 - acc: 0.8848 - val_loss: 0.3100 - val_acc: 0.9154
Epoch 5/50
35/35 [==============================] - 483s 14s/step - loss: 0.2403 - acc: 0.9192 - val_loss: 0.2803 - val_acc: 0.9118
Epoch 6/50
35/35 [==============================] - 473s 14s/step - loss: 0.2581 - acc: 0.9178 - val_loss: 0.2775 - val_acc: 0.9191
Epoch 7/50
35/35 [==============================] - 494s 14s/step - loss: 0.1853 - acc: 0.9361 - val_loss: 0.2446 - val_acc: 0.9449
Epoch 8/50
35/35 [==============================] - 505s 14s/step - loss: 0.

KeyboardInterrupt: 

In [53]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
#Retraining

In [98]:
# setting retraining from old model
model_path = "output/models/"+"weights-improvement-02-0.81.hdf5"
epochs=15
verbose=1

model = keras.models.load_model(model_path)

# retrain by loading last good model
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks_list)

Epoch 1/15
95/95 [==============================] - 1897s 20s/step - loss: 0.1716 - acc: 0.9482 - val_loss: 0.9000 - val_acc: 0.8129
Epoch 2/15
95/95 [==============================] - 1879s 20s/step - loss: 0.1657 - acc: 0.9488 - val_loss: 0.8720 - val_acc: 0.8176
Epoch 3/15
95/95 [==============================] - 1830s 19s/step - loss: 0.1370 - acc: 0.9559 - val_loss: 1.0008 - val_acc: 0.7933


In [22]:
# testing model
model_path = "output/models/"+"weights-improvement-01-0.81.hdf5"
model = keras.models.load_model(model_path)

In [ ]:
result = model.evaluate_generator(generator=test_generator)

In [ ]:
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

In [ ]:
# Best

# Accuracy: 84.51%
# Loss: 0.65